In [1]:
import requests
import json
# import openai
from openai import AzureOpenAI
import time
from dotenv import load_dotenv
import argparse, os

load_dotenv()

True

In [2]:
os.getenv("DEPLOYMENT_NAME")

'R21-GPT4'

In [3]:
test = """I broke 4 ribs,I was given 30 TylenolW/Cod. It did absolutely nothing for pain. Ibuprofen gave me a tiny bit of relief.i was surprised."""

In [4]:
context = """You are an AI assistant that helps to annotate social media comment data related to opioids and/or drug addiction, and you know everything about identifying the grammartical person (ie first, second, or third person) of a sentence or post"""

In [6]:
prompt = """I will give you an opioid-related social media comment to annotate. You will return a onehot integer of length 3 (eg. 010). If the comment contains a first-person or personal account with opioids put 1 in position 1 of the onehot integer else 0. If the comment contains a second-person experience with opioid, put 1 in position 2 of the onehot integer else 0. If the comment contains a third-person experience with opioid, put 1 in position 3 of the onehot integer else 0. Examples: return 100 for 'I am recovering from OUD', 010 for 'I am sorry to hear you lost your son to fentanyl OD', 001 for 'They take fent laced substances'. Note that some comments can have multiple persons-perspective and you will put 1 in the positions corresponding to the persons-perspective present. Example return 110 for 'I lost a partner. I am sorry to hear you lost someone too. My condolences to you and your family', and 011 for 'You should take lower dose. They should be careful with supplies from the street'. 
Annotate the following:\n""" + test

In [7]:
print(prompt)

I will give you an opioid-related social media comment to annotate. You will return a onehot integer of length 3 (eg. 010). If the comment contains a first-person or personal account with opioids put 1 in position 1 of the onehot integer else 0. If the comment contains a second-person experience with opioid, put 1 in position 2 of the onehot integer else 0. If the comment contains a third-person experience with opioid, put 1 in position 3 of the onehot integer else 0. Examples: return 100 for 'I am recovering from OUD', 010 for 'I am sorry to hear you lost your son to fentanyl OD', 001 for 'They take fent laced substances'. Note that some comments can have multiple persons-perspective and you will put 1 in the positions corresponding to the persons-perspective present. Example return 110 for 'I lost a partner. I am sorry to hear you lost someone too. My condolences to you and your family', and 011 for 'You should take lower dose. They should be careful with supplies from the street'. 


In [11]:
client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2024-02-01"
)

response = client.chat.completions.create(
    model=os.getenv("DEPLOYMENT_NAME"), # model = "deployment_name".
    messages=[
        {"role": "system", "content": context},
        {"role": "user", "content": prompt},
    ]
)

print(response.choices[0].message.content)

100


In [7]:
openai.api_key = os.getenv("AZURE_OPENAI_KEY")
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
openai.api_type = 'azure' # from quickstart documentation
openai.api_version = '2023-05-15' # from quickstart documentation

deployment_name = os.getenv("DEPLOYMENT_NAME")

# outname = "%s-output.txt" % args.drug

response = openai.ChatCompletion.create(
                engine = deployment_name,
                temperature = 0.7,
                messages=[
                            {"role": "system", "content": context},
                            {"role": "user", "content": prompt} 
                         ]
                    )

message = response['choices'][0]['message']['content']

APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [82]:
import requests
import json, math
# import openai
from openai import AzureOpenAI
import time
from dotenv import load_dotenv
import os, time, glob
from tqdm import tqdm
from statistics import mode
from sklearn.metrics import confusion_matrix
import pandas as pd

load_dotenv()

True

In [27]:
def query_o1(prompt):
    client = AzureOpenAI(
      azure_endpoint = os.getenv("o1_endpoint"), 
      api_key=os.getenv("o1_key"),  
      api_version="2024-02-01"
    )
    
    response = client.chat.completions.create(
        model=os.getenv("o1_mini"), # model = "deployment_name".
        messages=[
            {"role": "user", "content": prompt},
        ]
    )
    return response.choices[0].message.content

In [71]:
comments_path = "/oak/stanford/groups/rbaltman/samori/tiktok/comments/cleaned/comments_5_cleaned.csv"
output_path = ""
df = pd.read_csv(comments_path)
ids = list(df['id'])
comments = list(df['text'])

In [62]:
with open("./query_template.txt", "r") as f:
    query_template = f.read()

In [67]:
output_format =  "\n\nResponse format(never deviate):\n[Annotation]\n[Explanation]"

In [99]:
def annotate_comments(all_ids, comments):
    machine_id = 1
    num_machines = 2
    
    with open("./query_template.txt", "r") as f:
        query_template = f.read()
        
    output_format =  "\n\nResponse format(never deviate):\n[Annotation]\n[Explanation]"
    
    annotations = []
    reasonings = []
    ids = []
    
    for i in range(len(comments)):
        if i%num_machines == machine_id:
            comment = comments[i]
            comment_ids = all_ids[i]
        
            prompt = query_template + comment + output_format
            response = query_o1(prompt)
            
            annotation, reasoning = response.split("\n")

            ids.append(comment_ids)
            annotations.append(annotation)
            reasonings.append(reasoning)
            
            time.sleep(0.5)
    return ids, annotations, reasonings


def main(comments_path, output_path):
    machine_id = 1
    
    df = pd.read_csv(comments_path)
    
    all_ids = list(df['id'])
    comments = list(df['text'])

    all_ids = all_ids[:10]
    comments = comments[:10]

    ids, annotations, reasonings = annotate_comments(all_ids, comments)
        
    gpt_df = pd.DataFrame()
    gpt_df["id"] = ids
    gpt_df["annotation"] = annotations
    gpt_df["reasoning"] = reasonings
    gpt_df.to_csv(f"{output_path}/gpt_annotation_machine_{machine_id}.csv", index=None)

In [100]:
main(comments_path,"./")

In [69]:
gpt_df = pd.DataFrame()
gpt_df["id"] = ids[:10]
gpt_df["annotation"] = annotations
gpt_df["reasoning"] = reasonings
gpt_df.to_csv(f"{output_path}/machine_{x}.csv", index=None)

In [84]:
num_samples = math.ceil(len(df)/10)


In [78]:
list(df.loc[11:20]['text'])

['So heartbreaking 💔 prayers and thoughts 😔❤🙏',
 'And we have to acknowledge that there is a risk every time you allow a doctor to inject anything into your spine.',
 'It makes me so mad that they’re pushing extremely expensive, non-FDA approved, non-effective injections. It’s about the money and it’s wrong.',
 'Isn’t it so bad? It time just just encourage and empower all of our pain people the best way we can. And help them avoid the big ones that are realbad',
 'They also make you sign away your ability to sue them. I gotta find their intake forms. 🥺',
 'Exactly ~ it’s clearly noted on google, FDA, ….they caused more harm than good for me.',
 'It’s been rough. We’ve been in a heat dome here, but I’m still struggling with high pain. Be patient with yourself! 🥰🥰🥰',
 'injections made my pain worse',
 'Thank you Kim for all the information you give us!🥰',
 'They also all contain mercury as the preservative. I asked my Dr. for the insert of the injection and pointed out why I was refusing

In [89]:
10%10

0

In [70]:
gpt_df

id annotation                                          reasoning
0  7.295967e+18        001  The comment primarily describes others ("many ...
1  7.295968e+18        010  The comment directly addresses another individ...
2  7.295967e+18        100  The comment centers on the speaker's own pain ...
3  7.187482e+18        010  The comment directly addresses Jackson using s...
4  7.203023e+18        100  The comment primarily reflects the speaker’s o...
5  7.210254e+18        110  The comment centers on the speaker’s personal ...
6  7.210255e+18        100  The comment centers on the speaker’s personal ...
7  7.211994e+18        010  The comment directly addresses Jackson (using ...
8  7.207786e+18        110  The comment primarily centers on the speaker's...
9  7.215409e+18        100  The comment conveys the speaker's personal emo...

In [66]:
print(response)

**000**

**Explanation:**  
The comment primarily focuses on the speaker’s personal experience of loss, mentioning the death of her husband and the greater difficulty in losing her son. However, there is no explicit reference to drug-related issues or experiences within the comment. Since the main message centers on personal grief without directly addressing drug use, recovery, overdose, or related topics, none of the specified drug-related perspectives (first, second, or third person) are central to the comment’s content.


In [49]:
ids = list(df['id'])
comments = list(df['text'])
comment = comments[0]

In [56]:
prompt = query_template + comment + output_format

In [57]:
print(prompt)

I will give you a social media comment to annotate. This comment is part of a larger drugs discussion (e.g., about opioids, etc). Your task is to determine the perspective(s) expressed about drugs—specifically focusing on the main, central drug-related message of the comment. When considering perspectives, take into account not only the presence of pronouns, but more importantly, whether that perspective is central to the comment’s description of drug use, recovery, overdose loss, or other drug-related experiences. Do not automatically register a perspective simply because a pronoun appears; ensure that its use meaningfully contributes to the drug-related context.

You must return a onehot integer of length 3 (e.g., 010) corresponding to the following perspectives:
	- Position 1 (first person): Mark 1 if the comment’s main drug-related message comes from the speaker’s own experience or personal account (e.g., “I am recovering from OUD” or “I’ve taken every opiate under the sun”). Other

In [51]:
respone = query_o1(prompt)
annotation, reasoning = respone.split("\n")
annotation

ValueError: too many values to unpack (expected 2)

In [58]:
annotations = []
reasonings = []

for i in range(10):
    comment = comments[i]
    comment_id = ids[i]

    prompt = query_template + comment + output_format
    response = query_o1(prompt)
    print(comment)
    print(response)
    annotation, reasoning = response.split("\n")

    annotations.append(annotation)
    reasonings.append(reasoning)

many of them have no idea they are using fentanyl!
001
The comment primarily describes others' behavior by stating "many of them have no idea they are using fentanyl," which centers on third-person perspectives. There are no first-person ("I") or second-person ("you") references that contribute to the main drug-related message.
your children are spiritual warefare!
010
The comment directly addresses another individual using the second-person pronoun "your," focusing on their children in the context of spiritual warfare. This direct address is central to the drug-related discussion, likely implying a struggle related to the children's involvement with drugs. There is no reference to the speaker’s own experience (first person) or to others outside the addressed individual (third person).
i pray they never havevto understand our pain
100
The comment centers on the speaker's own emotions and experiences, as indicated by "I pray" and "our pain." While it references "they," this external gro

In [59]:
gpt_df = pd.DataFrame()
gpt_df["id"] = ids[:10]
gpt_df["annotation"] = annotations
gpt_df["reasoning"] = reasonings

In [60]:
gpt_df

id annotation                                          reasoning
0  7.295967e+18        001  The comment primarily describes others' behavi...
1  7.295968e+18        010  The comment directly addresses another individ...
2  7.295967e+18        100  The comment centers on the speaker's own emoti...
3  7.187482e+18        010  The comment directly addresses Jackson using t...
4  7.203023e+18        001  The comment uses a third-person pronoun ("his"...
5  7.210254e+18        100  The comment primarily centers on the speaker's...
6  7.210255e+18        100  The comment centers on the speaker’s personal ...
7  7.211994e+18        010  The comment directly addresses Jackson using t...
8  7.207786e+18        110  The comment primarily centers on the speaker’s...
9  7.215409e+18        100  The comment expresses the speaker’s personal f...

In [2]:
import openai

In [3]:
openai.BadRequestError

openai.BadRequestError